<a href="https://colab.research.google.com/github/erichsdb/Deep-Reinforcement-Learning/blob/main/notebooks/5-Bandits2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bandits - part 2

In the exercise, we will investigate in more details the properties of the bandit algorithms implemented last time and investigate more advanced ones (explained in the Sutton and Barto book).

**Q:** Start by copying all class definitions of the last exercise (Bandit, Greedy, $\epsilon$-Greedy, softmax) and re-run the experiments with correct values for the parameters in a single cell. We will ignore exploration scheduling (although we should not).

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import math

rng = np.random.default_rng()

class SoftmaxScheduleAgent:
    def __init__(self, bandit, alpha, t, tau_decay, q_init=0):
        self.bandit = bandit
        self.alpha = alpha
        # create an array filled with q_init
        self.Q_t = np.zeros(bandit.nb_actions)
        self.Q_t.fill(q_init)
        self.t = t
        self.tau_decay = tau_decay

        self.optimal_action = []

    def act(self):
        action = rng.choice(np.where(self.Q_t == self.Q_t.max())[0])
        return action

    def update(self, action, reward):
        self.Q_t[action] = self.Q_t[action] + self.alpha * (reward - self.Q_t[action])

    def train(self, nb_steps):
        for t in range(nb_steps):
          action = rng.choice(np.where(self.Q_t == self.Q_t.max())[0])

          # softmax selection
          prob = []

          for a in range(self.bandit.nb_actions):
            sum = 0
            for b in range(self.bandit.nb_actions):
                sum += np.exp(((self.Q_t[b] - self.Q_t.max())/self.t))
            prob.append(np.exp((self.Q_t[a] - self.Q_t.max()) / self.t) / sum )

          # chose reward based on probabilites
          action = rng.choice(range(self.bandit.nb_actions), p=prob)

          reward = self.bandit.step(action)
          self.update(action, reward)


          if a == self.bandit.a_star:
            self.optimal_action.append(1)
          else:
            self.optimal_action.append(0)

          if (self.t > self.tau_decay): self.t -= self.tau_decay

class SoftmaxAgent:
    def __init__(self, bandit, alpha, t, q_init):
        self.bandit = bandit
        self.alpha = alpha
        # create an array filled with q_init
        self.Q_t = np.zeros(bandit.nb_actions)
        self.Q_t.fill(q_init)
        self.t = t

        self.optimal_action = []

    def act(self):
        action = rng.choice(np.where(self.Q_t == self.Q_t.max())[0])
        return action

    def update(self, action, reward):
        self.Q_t[action] = self.Q_t[action] + self.alpha * (reward - self.Q_t[action])

    def train(self, nb_steps):
        for t in range(nb_steps):
          action = rng.choice(np.where(self.Q_t == self.Q_t.max())[0])

          # softmax selection
          prob = []

          for a in range(self.bandit.nb_actions):
            sum = 0
            for b in range(self.bandit.nb_actions):
                sum += np.exp(((self.Q_t[b] - self.Q_t.max())/self.t))
            prob.append(np.exp((self.Q_t[a] - self.Q_t.max()) / self.t) / sum )

          # chose reward based on probabilites
          action = rng.choice(range(self.bandit.nb_actions), p=prob)

          reward = self.bandit.step(action)
          self.update(action, reward)


          if a == self.bandit.a_star:
            self.optimal_action.append(1)
          else:
            self.optimal_action.append(0)

class EpsilonGreedyAgent:
    def __init__(self, bandit, alpha, epsilon, q_init=0):
        self.bandit = bandit
        self.alpha = alpha
        # create an array filled with q_init
        self.Q_t = np.zeros(bandit.nb_actions)
        self.Q_t.fill(q_init)
        self.epsilon = epsilon

        self.optimal_action = []

    def act(self):
        action = rng.choice(np.where(self.Q_t == self.Q_t.max())[0])
        return action

    def update(self, action, reward):
        self.Q_t[action] = self.Q_t[action] + self.alpha * (reward - self.Q_t[action])

    def train(self, nb_steps):
        for t in range(nb_steps):
          action = rng.choice(np.where(self.Q_t == self.Q_t.max())[0])

          # eta selection
          if rng.random() < self.epsilon:
            action = rng.choice([i for i in range(self.bandit.nb_actions) if i != action])

          reward = self.bandit.step(action)
          self.update(action, reward)


          if action == self.bandit.a_star:
            self.optimal_action.append(1)
          else:
            self.optimal_action.append(0)

class GreedyAgent:
    def __init__(self, bandit, alpha, q_init):
        self.bandit = bandit
        self.alpha = alpha
        self.Q_t = np.zeros(bandit.nb_actions)
        self.Q_t.fill(q_init)

        self.optimal_action = []

    def act(self):
        action = rng.choice(np.where(self.Q_t == self.Q_t.max())[0])
        return action

    def update(self, action, reward):
        self.Q_t[action] = self.Q_t[action] + self.alpha * (reward - self.Q_t[action])

    def train(self, nb_steps):
        for t in range(nb_steps):
          action = rng.choice(np.where(self.Q_t == self.Q_t.max())[0])
          reward = self.bandit.step(action)
          self.update(action, reward)


          if action == self.bandit.a_star:
            self.optimal_action.append(1)
          else:
            self.optimal_action.append(0)

class Bandit:
    """
    n-armed bandit.
    """
    def __init__(self, nb_actions, mean=0.0, std_Q=1.0, std_r=1.0):
        """
        :param nb_actions: number of arms.
        :param mean: mean of the normal distribution for $Q^*$.
        :param std_Q: standard deviation of the normal distribution for $Q^*$.
        :param std_r: standard deviation of the normal distribution for the sampled rewards.
        """
        # Store parameters
        self.nb_actions = nb_actions
        self.mean = mean
        self.std_Q = std_Q
        self.std_r = std_r

        # Initialize the true Q-values
        self.Q_star = rng.normal(self.mean, self.std_Q, self.nb_actions)

        # Optimal action
        self.a_star = self.Q_star.argmax()

    def step(self, action):
        """
        Sampled a single reward from the bandit.

        :param action: the selected action.
        :return: a reward.
        """
        return float(rng.normal(self.Q_star[action], self.std_r, 1)[0])

## Reward distribution

We are now going to vary the reward distributions and investigate whether the previous experimental results still hold when the true Q-values are in $\mathcal{N}(0, 1)$ and the rewards have a variance of 1.

**Q:** Let's now change the distribution of true Q-values from $\mathcal{N}(0, 1)$ to $\mathcal{N}(10, 10)$ when creating the bandits and re-run the algorithms. What happens and why? Modify the values of `epsilon` and `tau` to try to get a better behavior.

In [7]:
nb_actions = 5

# values from 0 - 10
mean_values = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

for mean in mean_values:
  rewards_epsilon_softmax = []
  optimal_epsilon_softmax = []

  for i in range(200):
    bandit = Bandit(nb_actions, mean=0, std_Q=mean, std_r=mean)
    agent = SoftmaxScheduleAgent(bandit, alpha=0.1, t=1, tau_decay=0.01)
    agent.train(200)

    rewards_epsilon_softmax.append(agent.Q_t)
    optimal_epsilon_softmax.append(1) if agent.act() == bandit.a_star else optimal_epsilon_softmax.append(0)

  rewards_epsilon_softmax = np.mean(rewards_epsilon_softmax, axis=0)
  optimal_epsilon_softmax = np.mean(optimal_epsilon_softmax, axis=0)

  print(f"{mean}: {rewards_epsilon_softmax}")
  print(f"{mean}: {optimal_epsilon_softmax}")
  # result is decreasing fast the higher the mean gets
  # result is relatively stable the higher the std gets --> (0,2) is actually good

0: [0. 0. 0. 0. 0.]
0: 0.245
1: [0.09717083 0.00880566 0.09656495 0.10559485 0.11674778]
1: 0.865
2: [0.24179169 0.31791526 0.12978829 0.27649875 0.34898556]
2: 0.895
3: [0.54464035 0.29473889 0.27222837 0.59921228 0.31295425]
3: 0.85
4: [0.80743012 0.22134167 0.47376885 0.61987232 0.32861217]
4: 0.835
5: [0.40553414 0.31255249 0.59883902 0.93416795 0.39303285]
5: 0.825
6: [0.33247188 0.83521469 0.79207664 0.56063425 0.83384439]
6: 0.78
7: [0.5977077  0.53009592 0.62455926 0.94059312 0.38791304]
7: 0.815
8: [1.07996162 1.09486432 0.68754277 1.42941    0.60519221]
8: 0.805
9: [0.59506235 0.81976838 1.22108042 0.89868732 1.49713611]
9: 0.78


## Optimistic initialization

The initial estimates of 0 are now very **pessimistic** compared to the average reward you can get (between 10 and 20). This was not the case in the original setup.

**Q:** Modify the classes so that they accept a parameter `Q_init` allowing to initialize the estimates `Q_t` to something different from 0. Change the initial value of the estimates to 10 for each algorithm. What happens? Conclude on the importance of reward scaling.

In [26]:
nb_actions = 5

rewards_epsilon_softmax = []
optimal_epsilon_softmax = []

for i in range(200):
  bandit = Bandit(nb_actions, mean=10, std_Q=10, std_r=10)
  agent = SoftmaxAgent(bandit, alpha=0.1, t=1, q_init=10)
  agent.train(200)

  rewards_epsilon_softmax.append(agent.Q_t)
  optimal_epsilon_softmax.append(1) if agent.act() == bandit.a_star else optimal_epsilon_softmax.append(0)

rewards_epsilon_softmax = np.mean(rewards_epsilon_softmax, axis=0)
optimal_epsilon_softmax = np.mean(optimal_epsilon_softmax, axis=0)

print(f"{10}: {rewards_epsilon_softmax}")
print(f"{10}: {optimal_epsilon_softmax}")

# result is depending on the Q-init value --> closer to the real value --> better result

10: [12.02142187 11.35868782 11.9686761  10.90397739 10.98431892]
10: 0.83


Let's now use **optimistic initialization**, i.e. initialize the estimates to a much higher value than what is realistic.

**Q:** Implement optimistic initialization by initializing the estimates of all three algorithms to 25. What happens?

In [33]:
nb_actions = 5
q_init = 50

print("Softmax Agent")

rewards_epsilon_softmax = []
optimal_epsilon_softmax = []

for i in range(200):
  bandit = Bandit(nb_actions, mean=10, std_Q=10, std_r=10)
  agent = SoftmaxAgent(bandit, alpha=0.1, t=1, q_init=q_init)
  agent.train(200)

  rewards_epsilon_softmax.append(agent.Q_t)
  optimal_epsilon_softmax.append(1) if agent.act() == bandit.a_star else optimal_epsilon_softmax.append(0)

rewards_epsilon_softmax = np.mean(rewards_epsilon_softmax, axis=0)
optimal_epsilon_softmax = np.mean(optimal_epsilon_softmax, axis=0)

print(f"{10}: {rewards_epsilon_softmax}")
print(f"{10}: {optimal_epsilon_softmax}")

print("Epsilon Greedy Agent")

rewards_epsilon_softmax = []
optimal_epsilon_softmax = []

for i in range(200):
  bandit = Bandit(nb_actions, mean=10, std_Q=10, std_r=10)
  agent = EpsilonGreedyAgent(bandit, alpha=0.1, epsilon=0.1, q_init=q_init)
  agent.train(200)

  rewards_epsilon_softmax.append(agent.Q_t)
  optimal_epsilon_softmax.append(1) if agent.act() == bandit.a_star else optimal_epsilon_softmax.append(0)

rewards_epsilon_softmax = np.mean(rewards_epsilon_softmax, axis=0)
optimal_epsilon_softmax = np.mean(optimal_epsilon_softmax, axis=0)

print(f"{10}: {rewards_epsilon_softmax}")
print(f"{10}: {optimal_epsilon_softmax}")

print("Greedy Agent")

rewards_epsilon_softmax = []
optimal_epsilon_softmax = []

for i in range(200):
  bandit = Bandit(nb_actions, mean=10, std_Q=10, std_r=10)
  agent = GreedyAgent(bandit, alpha=0.1, q_init=q_init)
  agent.train(200)

  rewards_epsilon_softmax.append(agent.Q_t)
  optimal_epsilon_softmax.append(1) if agent.act() == bandit.a_star else optimal_epsilon_softmax.append(0)

rewards_epsilon_softmax = np.mean(rewards_epsilon_softmax, axis=0)
optimal_epsilon_softmax = np.mean(optimal_epsilon_softmax, axis=0)

print(f"{10}: {rewards_epsilon_softmax}")
print(f"{10}: {optimal_epsilon_softmax}")


# somehow we saw a better result at a starting point that is too high

Softmax Agent
10: [16.74790221 17.30600229 17.37566872 17.23710656 16.94093117]
10: 0.835
Epsilon Greedy Agent
10: [16.03897558 16.414537   16.44114822 15.96470697 16.44338848]
10: 0.825
Greedy Agent
10: [16.80129973 17.13688241 17.13887764 17.21683183 16.89081345]
10: 0.84


## Reinforcement comparison

The problem with the previous **value-based** methods is that the Q-value estimates depend on the absolute magnitude of the rewards (by definition). The hyperparameters of the learning algorithms (learning rate, exploration, initial values) will therefore be very different depending on the scaling of the rewards (between 0 and 1, between -100 and 100, etc).

A way to get rid of this dependency is to introduce **preferences** $p_t(a)$ for each action instead of the estimated Q-values. Preferences should follow the Q-values: an action with a high Q-value should have a high Q-value and vice versa, but we do not care about its exact scaling.

In **reinforcement comparison**, we introduce a baseline $\tilde{r}_t$ which is the average received reward **regardless the action**, i.e. there is a single value for the whole problem. This average reward is simply updated after each action with a moving average of the received rewards:

$$\tilde{r}_{t+1} = \tilde{r}_{t} + \alpha \, (r_t - \tilde{r}_{t})$$

The average reward is used to update the preference for the action that was just executed:

$$p_{t+1}(a_t) = p_{t}(a_t) + \beta \, (r_t - \tilde{r}_{t})$$

If the action lead to more reward than usual, its preference should be increased (good surprise). If the action lead to less reward than usual, its preference should be decreased (bad surprise).

Action selection is simply a softmax over the preferences, without the temperature parameter (as we do not care about the scaling):

$$
    \pi (a) = \frac{\exp p_t(a)}{ \sum_b \exp p_t(b)}
$$

**Q:** Implement reinforcement comparison (with $\alpha=\beta=0.1$) and compare it to the other methods on the default settings.

In [40]:
class ReinforcementComparisonAgent:
    def __init__(self, bandit, alpha, beta, q_init):
        self.bandit = bandit
        self.alpha = alpha
        # create an array filled with q_init
        self.Q_t = np.zeros(bandit.nb_actions)
        self.Q_t.fill(q_init)

        self.baseline = 0
        self.preference = np.zeros(bandit.nb_actions)

        self.optimal_action = []

    def act(self):
        action = rng.choice(np.where(self.Q_t == self.Q_t.max())[0])
        return action

    def update(self, action, reward):
        self.Q_t[action] = self.Q_t[action] + self.alpha * (reward - self.Q_t[action])

    def train(self, nb_steps):
        for t in range(nb_steps):
          action = rng.choice(np.where(self.Q_t == self.Q_t.max())[0])

          # softmax selection
          prob = []

          for a in range(self.bandit.nb_actions):
            sum = 0
            for b in range(self.bandit.nb_actions):
                sum += np.exp(self.preference[b])
            print(sum)
            print(np.exp(self.preference[a]))
            prob.append(np.exp(self.preference[a])) / sum

          # chose reward based on probabilites
          action = rng.choice(range(self.bandit.nb_actions), p=prob)

          reward = self.bandit.step(action)
          self.update(action, reward)

          # update the reinforcement comparison
          self.baseline = self.baseline + self.alpha * (reward - self.baseline)
          self.preference[action] = self.preference[action] + self.beta * (reward - self.baseline)

          if a == self.bandit.a_star:
            self.optimal_action.append(1)
          else:
            self.optimal_action.append(0)



**Q:** Compare all methods with optimistic initialization. The true Q-values come from $\mathcal{N}(10, 10)$, the estimated Q-values are initialized to 20 for greedy, $\epsilon$-greedy and softmax, and the average reward is initialized to 20 for RC (the preferences are initialized at 0).  

In [41]:
nb_actions = 5
q_init = 50

print("Softmax Agent")

rewards_epsilon_softmax = []
optimal_epsilon_softmax = []

for i in range(200):
  bandit = Bandit(nb_actions, mean=10, std_Q=10, std_r=10)
  agent = ReinforcementComparisonAgent(bandit, alpha=0.1, beta=0.1, q_init=q_init)
  agent.train(200)

  rewards_epsilon_softmax.append(agent.Q_t)
  optimal_epsilon_softmax.append(1) if agent.act() == bandit.a_star else optimal_epsilon_softmax.append(0)

rewards_epsilon_softmax = np.mean(rewards_epsilon_softmax, axis=0)
optimal_epsilon_softmax = np.mean(optimal_epsilon_softmax, axis=0)

print(f"{10}: {rewards_epsilon_softmax}")
print(f"{10}: {optimal_epsilon_softmax}")

Softmax Agent
5.0
1.0


TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'